In [1]:
import path_configs # noqa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import settings
settings.init()
from modules.EncoderGenerators import * # noqa

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# assumes that array is not zero
def scaled(tensor):
    return (tensor-tf.math.reduce_min(tensor))/(tf.math.reduce_max(tensor)-tf.
                                                math.reduce_min(tensor))

TRAIN_DATA_LEN: 3720334
VAL_DATA_LEN: 1593071


In [7]:
from DataPreprocessor import DataLoader
dl = DataLoader()
X_array = dl.make_spectrogram(1241)

In [8]:
X_array.shape

TensorShape([1025, 5261])

In [14]:
frame_step = 64
np.ceil( (max(settings.LENGTHS)//frame_step - (frame_step//2) + 1)/1024)*1024

8192.0

In [130]:
class TrainClassifierGenerator(tf.data.Dataset):
    def _generator(case_nums, TIMESTEPS, WINDOWS_STEP, BATCH_SIZE):
        autoencoder = tf.keras.models.load_model('model_autoencoder_final.h5')
        encoder = autoencoder.layers[1]
        encoder = tf.keras.models.Model(encoder.layers[0].input, encoder.layers[2]
                                        .output)

        from DataPreprocessor import DataLoader
        dl = DataLoader()
        data_len = 0
        max_batch_len = BATCH_SIZE*TIMESTEPS
        audio_batch = []
        while True:
            for case_num in case_nums:
                X_sample = dl.make_spectrogram(case_num)
                data_len += X_sample.shape[1]
                if data_len > max_batch_len:
                    # deleting silence
                    X_array = np.array(np.column_stack(audio_batch))
                    silence = np.argwhere(np.all(X_array[..., :] == -80, axis=0))            
                    X_array = tf.cast(np.swapaxes(np.delete(X_array, silence, axis=1),
                                                0, 1), tf.float32)
                    # padding to smallest n*timestep above max length
                    frame_step = 64
                    padding = int(max_batch_len - X_array.shape[0])
                    X_array = tf.pad(X_array, ([0,padding], [0, 0]))
                    X_array = tf.cast(X_array, tf.float32)
                    X_array = tf.reshape(X_array, [1024, 64, 1025])
                    latent_spectral_frames = encoder.predict(X_array)
                    yield(latent_spectral_frames)
                    audio_batch = []

                audio_batch.append(X_sample)


    def __new__(cls,
                case_nums=settings.AE_TRAIN_IDX,
                TIMESTEPS=settings.AE_TIMESTEPS,
                WINDOWS_STEP=settings.AE_WINDOWS_STEP,
                N_FEATURES=settings.AE_N_FEATURES,
                BATCH_SIZE=settings.MODEL_BATCH_SIZE):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=tf.dtypes.float32,
            output_shapes=((BATCH_SIZE, 128)),
            args=(case_nums, TIMESTEPS, WINDOWS_STEP, BATCH_SIZE)
        )

In [131]:
counter = 0
for x in TrainClassifierGenerator():
    print(x.shape)
    counter+=1
    if counter>4:
        break

(1024, 128)
(1024, 128)
(1024, 128)
(1024, 128)
(1024, 128)


In [16]:
(4127/64 - (64/2) + 1)

33.484375

In [29]:
import pandas as pd
df = pd.read_excel('do_import_magazyn_19_09_2020.xlsx')

In [53]:
df['Destination Location Zone / ID bazy danych'].unique()

array([1353., 1354.,   22.,  943.])

In [74]:
df[df['Destination Location Zone / ID bazy danych']==1354].drop_duplicates(subset=['Stock Moves / Product'])

,ob_TowId,Stock Moves / Product,Stock Moves / Quantity,Stock Moves / Unit of Measure,Stock Moves / Description,Source Location Zone / ID bazy danych,Destination Location Zone / ID bazy danych,Picking Type / ID bazy danych,Symbol,Stock Moves / Unit price
3560,39,Singles GRA,1.0,Sztuki,Singles GRA,8.0,1354.0,1.0,000051,1.00
3561,108,FireWire na PCI,1.0,Sztuki,FireWire na PCI,NaN,1354.0,NaN,000126,12.31
3562,112,"Kieszeń USB SATA 3,5 Gembird",1.0,Sztuki,"Kieszeń USB SATA 3,5 Gembird",NaN,1354.0,NaN,000130,34.00
3563,136,"Intel Celeron 420 1,6 GHz LGA775",1.0,Sztuki,"Intel Celeron 420 1,6 GHz LGA775",NaN,1354.0,NaN,000155,0.00
3564,156,mysz A4-Tech BW-26,1.0,Sztuki,mysz A4-Tech BW-26,NaN,1354.0,NaN,000161(1)(4)(1)(1(3),19.00
...,...,...,...,...,...,...,...,...,...,...
5339,14908,APACER Pamięć DDR4 8GB 2400MHz CL17 SODIMM 1.2V,1.0,Sztuki,APACER Pamięć DDR4 8GB 2400MHz CL17 SODIMM 1.2V,NaN,1354.0,NaN,019209,97.51
5340,14928,"SWITCH LiteWave LS105G, 5 portów RJ45 10/100/1...",1.0,Sztuki,"SWITCH LiteWave LS105G, 5 portów RJ45 10/100/1...",NaN,1354.0,NaN,019235,46.94
5341,14962,APACER AP240GAS340G-1 Apacer SSD 240GB,3.0,Sztuki,APACER AP240GAS340G-1 Apacer SSD 240GB,NaN,1354.0,NaN,019290,84.62
5342,15088,ADAPTER VGA(M)->DVI-I(F)(24+5) CZARNY GEMBIRD,9.0,Sztuki,ADAPTER VGA(M)->DVI-I(F)(24+5) CZARNY GEMBIRD,NaN,1354.0,NaN,019438,2.17
